# Function Fitting

In [1]:
import numpy as np
from scipy.optimize import curve_fit
from scipy.optimize import least_squares
import DC_Pickle as dcp
import Curve_Functions as cv
import matplotlib.pyplot as plt
%matplotlib inline

def disp_Data(x, y_true, y_pred, file_path, clt_num, cost, rows=1, columns=1, size=(6, 4)):
    fig, ax = plt.subplots(rows, columns, figsize=size)
    ax.plot(x, y_true, 'rx', label='average score')
    ax.plot(x, y_pred, 'b-', label='curve fitting')
    ax.set_xlim([0, max(x)+1])
    ax.set_ylim([0, max(y_true)+0.2])
    ax.legend(fontsize=14)
    ax.set_title("cluster {0}: cost {1}".format(clt_num, round(cost, 2)))
    fig.savefig(file_path, dpi=100)
    plt.show()

## load data

In [2]:
train_idx = dcp.open_Pickle("../../data/pickles/clusters_origin/indices/index13.pickle")
train_idx = train_idx[9] #2, 8

train_scores = dcp.open_Pickle('../../data/pickles/seperate_origin/eventValue.pickle')
train_scores = train_scores[:300, :]/1e+4
attempts15 = np.arange(15)+1
attempts300 = np.arange(300)+1

## filtering discontinuous under 15 attempts
idx_all = []
idx_pure = []

for i in range(train_scores.shape[1]):
    if not np.isnan(train_scores[:15, i]).any():
        idx_all.append(i)
        idx_pure.append(i)
    else:
        idx_all.append(np.nan)

train_scores = train_scores[:, idx_pure]
print("Training data set: {0}".format(np.shape(train_scores)))

Training data set: (300, 22832)


In [3]:
## get cluster data and cluster average data (average is centroid)
nClt = 13
for i in range(nClt):
    name = "cluster{0}".format(i+1)
    globals()[name] = train_scores[:, train_idx==i] # get cluster data
    
    ## get cluster average data
    data = np.ones(300)
    for j in range(len(data)):
        avg = eval(name)[j, :]
        avg = np.sum(avg[~np.isnan(avg)])/len(avg[~np.isnan(avg)])
        data[j] = avg

    globals()["avg{0}".format(i+1)] = data

## make folder for each function

In [ ]:
dcp.make_folders("Figs/curve_fitting/")

## exponential fit

### Two parameters

#### train on 15 attempts

In [ ]:
dcp.make_folders("Figs/curve_fitting/exponential2")

seed = [1, 1]
for i in range(nClt):
    print("cluster {0}:".format(i+1))
    ## train
    exp2_opt, exp2_cost = cv.curve_Fitting(
        cv.exponential_least2, cv.exponential_curve2, 
        attempts15, eval("avg{0}".format(i+1))[:15], seed, 
        "Figs/curve_fitting/exponential2/{0}".format(i+1), clt_num = i+1)
    ## validation
    y_fit = cv.exponential_curve2(attempts300, exp2_opt[0], exp2_opt[1])
    
    exp2_cost300 = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit) # get cost for all data
    
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit, 
              file_path="Figs/curve_fitting/exponential2/valid{0}".format(i), 
              clt_num=i+1, cost = exp2_cost300)

### Three parameters

#### train on 15 attempts

In [ ]:
dcp.make_folders("Figs/curve_fitting/exponential3")

seed = [1, 1, 1]
for i in range(nClt):
    print("cluster {0}:".format(i+1))
    exp3_opt, exp3_cost = cv.curve_Fitting(
        cv.exponential_least3, cv.exponential_curve3, 
        attempts15, eval("avg{0}".format(i+1))[:15], seed,
        "Figs/curve_fitting/exponential3/{0}".format(i+1), clt_num = i+1)

    y_fit = cv.exponential_curve3(attempts300, exp3_opt[0], exp3_opt[1], exp3_opt[2])
    exp3_cost300 = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit) # get cost for all data
    
    cost300 = cv.cost_Function(attempts300, y_fit)
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit, 
              file_path="Figs/curve_fitting/exponential3/valid{0}".format(i), 
              clt_num=i+1, cost = exp3_cost300)

### polynoimial fit

In [ ]:
dcp.make_folders("Figs/curve_fitting/polynomial2")

seed = [1, 1]
for i in range(nClt):
    print("cluster {0}:".format(i+1))
    poly_opt, poly_cost = cv.curve_Fitting(
        cv.polynomial_least, cv.polynomial_curve, 
        attempts15, eval("avg{0}".format(i+1))[:15], seed,
        "Figs/curve_fitting/polynomial2/{0}".format(i+1), clt_num = i+1)

    y_fit = cv.polynomial_curve(attempts300, poly_opt[0], poly_opt[1])
    
    poly_cost300 = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit) # get cost for all data
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit, 
              file_path="Figs/curve_fitting/polynomial2/valid{0}".format(i), 
              clt_num=i+1, cost = poly_cost300)

### power law fit

#### Two parameters

In [ ]:
dcp.make_folders("Figs/curve_fitting/powerlaw2")

seed = [1, 1]
for i in range(nClt):
    print("cluster {0}:".format(i+1))
    pl2_opt, pl2_cost = cv.curve_Fitting(
        cv.powerlaw_least2, cv.powerlaw_curve2, 
        attempts15, eval("avg{0}".format(i+1))[:15], seed,
        "Figs/curve_fitting/powerlaw2/{0}".format(i+1), clt_num = i+1)
    
    y_fit = cv.powerlaw_curve2(attempts300, pl2_opt[0], pl2_opt[1])
    pl2_cost300 = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit) # get cost for all data
    
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit, 
              file_path="Figs/curve_fitting/powerlaw2/valid{0}".format(i), 
              clt_num=i+1, cost = pl2_cost300)

#### Thress parameters

In [ ]:
dcp.make_folders("Figs/curve_fitting/powerlaw3")

seed = [1, 1, 1]
for i in range(nClt):
    print("cluster {0}:".format(i+1))
    pl3_opt, pl3_cost = cv.curve_Fitting(
        cv.powerlaw_least3, cv.powerlaw_curve3, 
        attempts15, eval("avg{0}".format(i+1))[:15], seed,
        "Figs/curve_fitting/powerlaw3/{0}".format(i+1), clt_num = i+1)
    
    y_fit = cv.powerlaw_curve3(attempts300, pl3_opt[0], pl3_opt[1], pl3_opt[2])
    pl3_cost300 = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit) # get cost for all data
    
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit, 
              file_path="Figs/curve_fitting/powerlaw3/valid{0}".format(i), 
              clt_num=i+1, cost = pl3_cost300)

#### Four parameters

In [ ]:
dcp.make_folders("Figs/curve_fitting/powerlaw4")

seed = [1, 1, 1, 1]
for i in range(nClt):
    print("cluster {0}:".format(i+1))
    pl4_opt, pl4_cost = cv.curve_Fitting(
        cv.powerlaw_least4, cv.powerlaw_curve4, 
        attempts15, eval("avg{0}".format(i+1))[:15], seed,
        "Figs/curve_fitting/powerlaw4/{0}".format(i+1), clt_num = i+1)
    
    y_fit = cv.powerlaw_curve4(attempts300, pl4_opt[0], pl4_opt[1], pl4_opt[2], pl4_opt[3])
    pl4_cost300 = cv.cost_Function(eval("avg{0}".format(i+1)), y_fit) # get cost for all data
    
    disp_Data(attempts300, eval("avg{0}".format(i+1)), y_fit, 
              file_path="Figs/curve_fitting/powerlaw4/valid{0}".format(i), 
              clt_num=i+1, cost = pl4_cost300)

## Multiple curves

### exponential with three parameter

In [ ]:
seed = [1, 1, 1]

for clt_num in range(13):
    idx, cost, p1, p2 = cv.multi_Fitting2(cv.exponential_least3, clt_num, 
                                     eval("avg{0}".format(clt_num+1)), seed, n_param=3)
    
    y_mean1 = cv.exponential_curve3(attempts300, p1[0], p1[1], p1[2])
    y_mean2 = cv.exponential_curve3(attempts300, p2[0], p2[1], p2[2])
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    
    ax.plot(attempts300, avg11, 'rx', label='average score')
    ax.plot(attempts300[:idx], y_mean1[:idx], 'b-', label='curve 1', linewidth=3)
    ax.plot(attempts300[idx-50:], y_mean2[idx-50:], 'g-', label='curve 2', linewidth=3)
    
    ax.set_ylim([0, max(avg11)+0.2])
    ax.legend(fontsize=14)
    ax.set_title("cluster {0}: cost {1}".format(clt_num+1, round(cost, 2)))
    
    plt.show()
    
'''
y_mean1 = cv.powerlaw_curve4(attempts300, param1[0], param1[1], param1[2], param1[3])
y_mean2 = cv.powerlaw_curve4(attempts300, param2[0], param2[1], param2[2], param2[3])
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
'''
#ax.set_xlim([0, max()+1])

#fig.savefig(file_path, dpi=100)


In [ ]:
seed = [1, 1, 1]
idx1, idx2, cost, p1, p2, p3 = cv.multi_curveFitting_3(cv.exponential_least3, avg11, seed, n_param=3)

In [ ]:
y_mean1 = cv.exponential_curve3(attempts300, p1[0], p1[1], p1[2])
y_mean2 = cv.exponential_curve3(attempts300, p2[0], p2[1], p2[2])
y_mean3 = cv.exponential_curve3(attempts300, p3[0], p3[1], p3[2])

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.plot(attempts300, avg11, 'rx', label='average score')
ax.plot(attempts300[:idx1], y_mean1[:idx1], 'b-', label='curve 1', linewidth=3)
ax.plot(attempts300[idx1:], y_mean2[idx1:], 'g-', label='curve 2', linewidth=3)
ax.plot(attempts300[idx2:], y_mean3[idx2:], 'c-', label='curve 3', linewidth=3)

ax.set_ylim([0, max(avg11)+0.2])
ax.legend(fontsize=14)
#ax.set_title("cluster {0}: cost {1}".format(clt_num+1, round(cost, 2)))

plt.show()

print(idx1, idx2)

In [4]:
aaa = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print(aaa[:5])
print(aaa[5:])
print(np.linspace(1, 2, 2))
seed = [1, 1, 1]
cv.multi_curveFitting_3(cv.exponential_least3, avg11, seed, n_param=3)

[1, 2, 3, 4, 5]
[6, 7, 8, 9, 10]
[ 1.  2.]
iter  1
x1:[ 1.], x2:[ 2.], x3:298
x1:[ 1.], x2:[ 2.  4.], x3:297
x1:[ 1.], x2:[ 2.   3.5  5. ], x3:296
x1:[ 1.], x2:[ 2.          3.33333333  4.66666667  6.        ], x3:295
x1:[ 1.], x2:[ 2.    3.25  4.5   5.75  7.  ], x3:294
x1:[ 1.], x2:[ 2.   3.2  4.4  5.6  6.8  8. ], x3:293
x1:[ 1.], x2:[ 2.          3.16666667  4.33333333  5.5         6.66666667  7.83333333
  9.        ], x3:292
x1:[ 1.], x2:[  2.           3.14285714   4.28571429   5.42857143   6.57142857
   7.71428571   8.85714286  10.        ], x3:291
x1:[ 1.], x2:[  2.      3.125   4.25    5.375   6.5     7.625   8.75    9.875  11.   ], x3:290
x1:[ 1.], x2:[  2.           3.11111111   4.22222222   5.33333333   6.44444444
   7.55555556   8.66666667   9.77777778  10.88888889  12.        ], x3:289
x1:[ 1.], x2:[  2.    3.1   4.2   5.3   6.4   7.5   8.6   9.7  10.8  11.9  13. ], x3:288
x1:[ 1.], x2:[  2.           3.09090909   4.18181818   5.27272727   6.36363636
   7.45454545   8.54545

KeyboardInterrupt: 